<a href="https://colab.research.google.com/github/Jeeunun/Pandas/blob/main/19_%EB%AC%B8%EC%9E%90%EC%97%B4_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문자열 데이터 전처리
## #01.Colab에 Mecab-kor-doc 설치하기
한글 형태소 분석을 수행할 경우 새로운 노트북 생성시마다 해줘야 한다.

In [1]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
!bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab_light_220429.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91
Receiving objects: 100% (138/138), 1.72 MiB | 27.44 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Installing konlpy.....
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 26.9 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-09-17 11:53:21--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuse

## #02. 패키지 참조

In [2]:
from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.text import Tokenizer

## #03. 문자열 토큰화

### 1. Token(=형태소)
- 문법적으로 더 이상 나눌 수 없는 언어요소.
- 영어의 경우 각 단어로 나누면 되지만 한글의 경우 복잡한 처리과정을 거쳐야 하기 때문에 별도의 라이브러리를 적용해야 한다.(konlpy, mecab 등)

### 2. 영어문장에 대한 토큰화
#### 토큰화 학습 데이터

In [3]:
train_text = ['You are the Best', 'You are the Nice']

#### 토큰화 객체 생성
- `num_words`: 밀집 표현의 최대 벡터 길이를 지정(=최대단어수)
- `oov_token` : 학습 결과를 적용할 때 데이터에 포함되지 않는 단어가 존재할 경우 대체할 단어.

In [4]:
tokenizer = Tokenizer(num_words = 10, oov_token='<OOV>')

### 토큰화 학습하기

In [5]:
# 토큰화 학습
tokenizer.fit_on_texts(train_text)

# 각 단어에 부여된 인덱스 번호 확인
print(tokenizer.word_index)

{'<OOV>': 1, 'you': 2, 'are': 3, 'the': 4, 'best': 5, 'nice': 6}


### 학습 결과 적용하기
토근화를 학습한 모델에 새로운 단어를 적용하여 밀집 벡터로 변환한다.

In [6]:
train_text = ['We are the Best', 'We are the Nice']
sequences = tokenizer.texts_to_sequences(train_text)
print(sequences)

[[1, 3, 4, 5], [1, 3, 4, 6]]


### 3. 한글 문장 토큰화
형태소 분석 엔진 Mecab 설치해야 한다.

#### 토큰화 학습 데이터

In [7]:
poem = """
흘러내린 머리카락이 흐린 호박빛 아래 빛난다.
난 유영한다. 차분하게 과거에 살면서 현재의 공기를 마신다.
가로등이 깜빡인다.
나도 깜빡여준다.
"""

#### 불용어(stopwords)목록
분석에서 제외할 불필요한 단어에 대한 목록.

개발자가 임의로 정하거나 불용어 사전을 웹에서 내려받아 사용할 수 있다.

In [8]:
stopwords = ['난', '나']

#### 형태소 분석을 통해 명사만 추출하기

In [9]:
mecab = Mecab()
nouns = mecab.nouns(poem)
print(nouns)

['머리카락', '호박', '빛', '아래', '난', '유영', '과거', '현재', '공기', '가로등', '나']


In [12]:
len(nouns)

11

#### 추출된 명사에서 불용어를 제외한 새로운 리스트 만들기

In [10]:
train_text = []

for i in nouns:
  if i not in stopwords:
    train_text.append(i)

train_text

['머리카락', '호박', '빛', '아래', '유영', '과거', '현재', '공기', '가로등']

#### 토큰화 수행

In [11]:
tokenizer = Tokenizer(num_words=len(nouns), oov_token='<OOV>')
tokenizer.fit_on_texts(train_text)
print(tokenizer.word_index)

{'<OOV>': 1, '머리카락': 2, '호박': 3, '빛': 4, '아래': 5, '유영': 6, '과거': 7, '현재': 8, '공기': 9, '가로등': 10}


## #04. 워드 임베딩(Word Embedding)
컴퓨터가 자연어를 이해하고, 효율적으로 처리하게 하기 위해서는 컴퓨터가 이해할 수 있도록 자연어를 적절히 변환할 필요가 있다.

워드 임베딩은 단어를 벡터로 표현하는 방법으로 단어를 희소 표현이 개선된 형태인 밀집 표현으로 변환한다.

### 1. 희소표현(원-핫 백터)

원- 핫 인코딩과 비슷한 방법

표현하고자 하는 단어의 인덱스 값만 1이고 나머지 인덱스에는 전부 0으로 표현되는 벡터 방법으로 원-핫 백터라고도 한다.

#### 예시문장 두 개
`머신러닝 공부는 재미있다.`

`머신러닝은 유용하다.`

#### 형태소 분석
명사만 추출

`머신러닝, 공부, 재미`

`머신러닝, 유용`

#### 토큰화
각 단어에 인덱스 적용
`0,1,2`

`0,3`

#### 희소표현
전체 단어의 수가 `4개`이므로 각각의 단어를 4개의 원소를 갖는 리스트 안에서 one-hot인코딩으로 표현한다.

`[[1,0,0,0],[0,1,0,0],[0,0,1,0]]`

`[[1,0,0,0],[0,0,0,1]]`

#### 희소표현의 한계

단어의 개수가 늘어나면 벡터의 차원이 한없이 커진다.

여러 문장을 통해 얻어진 단어가 10,000개이고 그 중에서 5개의 단어로 구성된 문장이라면 벡터의 총 길이는 50,000이 되게 된다.

그러므로 공간적인 낭비를 가져온다.

### 2. 밀집표현
희소표현의 반대

벡터의 차원을 단어 집합의 크기로 결정하지 않고 분석가가 설정한 임의의 값으로 모든 단어 벡터의 차원을 맞춘다.

`0`과 `1`이 아닌 실수를 원소로 갖는다.

#### 예시
희소표현 → `[1, 0, 0, 0]`

벡터의 차원을 2로 설정한 밀집표현 → `[1.8, -0.4]`

> 수치 값은 임의의 값이다. 이와 같은 식으로 실수 형태로 표현된다는 것을 표현한 것일 뿐 실제 정확한 값은 아님을 이해하자.

결과적으로 벡터의 차원이 조밀해졌다고 하여 밀집 벡터라고 부른다.

### 3. 워드 임베딩

단어를 밀집 벡터의 형태로 표현한 방법.

케라스에서는 **토큰화** 한 단어 벡터를 `Embedding()` 함수에 전달하여 학습층을 쌓음으로서 적용할 수 있다.

단어를 랜덤한 값을 갖는 밀집 벡터로 변환한 뒤에, 인공 신경망의 가중치를 학습하는 방식으로 단어 벡터를 학습한다.